# End to End Pure Streaming Data-Pipeline for Building Table Using Spark Structured Streaming on Databricks

###### Description: In this notebook we read building state rows from incoming csv files into a streamig dataframe, transform (clean, cast, rename) the data, add/update the latest state to a Databricks Delta table
###### Objective: (incoming csv files) --> "building_streamingDF" --> "results_df" --> "building_data"

In [0]:
import requests
import json
import optimus as op
import phonenumbers 
import re
import datetime
import time

from pyspark.sql.types import *
from pyspark.sql.functions import udf
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext, Row
from pyspark.sql.functions import unix_timestamp, from_unixtime
from pyspark.sql import functions as F
from pyspark.sql.window import Window as W
from pyspark.sql import DataFrame
from pyspark.sql.functions import lit
from pyspark.sql.functions import rank, col

In [0]:
# Schema for building JSON
building_schema = StructType([
            StructField("Building_id", IntegerType(), False),
            StructField("Building_name", StringType(), True),
            StructField("Landlord_id", IntegerType(), False),
            StructField("Address_line_1", StringType(), False),
            StructField("City", StringType(), False),
            StructField("Post_code", StringType(), True),
            StructField("Region", StringType(), True),
            StructField("event_time", TimestampType(), True)])

building_udf_schema = StructType([
            StructField("Building_name", StringType(), True),
            StructField("Landlord_id", IntegerType(), False),
            StructField("Address_line_1", StringType(), False),
            StructField("City", StringType(), False),
            StructField("Post_code", StringType(), True),
            StructField("Region", StringType(), True),
            StructField("event_time", TimestampType(), True)])

###### Description: Get building csv files as a streaming "building_streamingDF" and process it on the fly and get transformed stream "building_df"
###### Objective: (incoming csv files) --> "building_streamingDF" --> "building_df"

In [0]:
# Get building Steaming DataFrame from csv files

# streaming starts here by reading the input files 
building_Path = "/FileStore/apartment/building/inprogress/"
building_streamingDF = (
  spark
    .readStream
    .schema(building_schema)
    .option("maxFilesPerTrigger", "1")
    .option("header", "true")
    .option("multiLine", "true")
    .csv(building_Path)
)
# Clear invalid rows
building_df = building_streamingDF.select("*").where("Building_id IS NOT NULL")
# Instantiation of DataTransformer class:
transformer = op.DataFrameTransformer(building_df)
# Replace NA with 0's
transformer.replace_na(0.0, columns="*")
# Clear accents: clear_accents only from name column and not everywhere 
transformer.clear_accents(columns='*')
# Remove special characters:  From all Columns 
transformer.remove_special_chars(columns=['Building_name', 'Address_line_1', 'City', 'Post_code', 'Region'])

Out[ 5 ]: <optimus.df_transformer.DataFrameTransformer at 0x7fa6008c9e80>

##### This function parses the corresponding columns into a single column

In [0]:
def my_fun(Building_name, Landlord_id, Address_line_1, City, Post_code, Region, event_time):
  return zip(Building_name, Landlord_id, Address_line_1, City, Post_code, Region, event_time)

udf_Fun = udf(my_fun, ArrayType(building_udf_schema))

In [0]:
intermediate_df = ( building_df.withWatermark("event_time", "10 seconds")
            .groupBy("Building_id")
            .agg(F.collect_list("Building_name").alias("Building_name"),
                 F.collect_list("Landlord_id").alias("Landlord_id"),
                 F.collect_list("Address_line_1").alias("Address_line_1"),
                 F.collect_list("City").alias("City"),
                 F.collect_list("Post_code").alias("Post_code"),
                 F.collect_list("Region").alias("Region"),
                 F.collect_list("event_time").alias("event_time"), 
                 F.max("event_time").alias("latest_event_time"))
            .select("Building_id", 
                    F.explode(udf_Fun(F.column("Building_name"), 
                                      F.column("Landlord_id"), 
                                      F.column("Address_line_1"), 
                                      F.column("City"), 
                                      F.column("Post_code"),
                                      F.column("Region"), 
                                      F.column("event_time")))
                    .alias("data"), "latest_event_time"))

##### Filter the data where event_time is latest

In [0]:
results_df = (intermediate_df
              .select("Building_id", 
                      "data.Building_name", 
                      "data.Landlord_id", 
                      "data.Address_line_1", 
                      "data.City", 
                      "data.Post_code", 
                      "data.Region",
                      "data.event_time", 
                      "latest_event_time")
              .where("data.event_time=latest_event_time")).orderBy("Building_id")

##### Display final result
###### This result shows the latest state of all the unique Building_id

In [0]:
display(results_df)

Building_id,Building_name,Landlord_id,Address_line_1,City,Post_code,Region,event_time,latest_event_time
1,Goodwin-Dickinson,1,8 American Ash Hill,Philadelphia,19151,Pennsylvania,2018-11-27T06:33:41.000+0000,2018-11-27T06:33:41.000+0000
2,Schultz-Schiller,2,37 Lotheville Hill,Dayton,45403,Ohio,2017-03-01T13:17:00.000+0000,2017-03-01T13:17:00.000+0000
3,Bashirian-Dibbert,3,45253 Gulseth Place,Denton,76210,Texas,2017-08-31T07:27:45.000+0000,2017-08-31T07:27:45.000+0000
4,Marquardt-Spencer,4,5590 Carpenter Junction,Muncie,47306,Indiana,2017-10-15T03:20:58.000+0000,2017-10-15T03:20:58.000+0000
5,"Ebert, Haley and Steuber",5,9354 Butternut Road,Orange,92668,California,2018-05-03T19:14:42.000+0000,2018-05-03T19:14:42.000+0000
6,"Hilll, Runte and Trantow",6,4 Crownhardt Road,Colorado Springs,80935,Colorado,2017-06-13T07:15:04.000+0000,2017-06-13T07:15:04.000+0000
7,Gottlieb LLC,7,993 International Way,Bradenton,34282,Florida,2017-11-07T22:51:57.000+0000,2017-11-07T22:51:57.000+0000
8,Bergstrom-Nitzsche,8,3 Sutteridge Place,Lafayette,47905,Indiana,2018-07-21T09:53:54.000+0000,2018-07-21T09:53:54.000+0000
9,"Jenkins, McCullough and Schimmel",9,2526 Cardinal Road,Jacksonville,32209,Florida,2018-02-02T22:50:38.000+0000,2018-02-02T22:50:38.000+0000
10,"Kutch, Mueller and Erdman",10,82416 Lindbergh Point,Indianapolis,46239,Indiana,2018-03-08T14:30:55.000+0000,2018-03-08T14:30:55.000+0000


##### Below cells are optional if external functionality or storage is needed

###### Write the stream to a Databricks Delta table for storage

In [0]:
streaming_query = (results_df.writeStream
 .format("delta")
 .outputMode("complete")
 .option("mergeSchema", "true")
 .option("checkpointLocation", "/delta/apartment/building/_checkpoints/streaming-agg")
 .start("/delta/apartment/building_data"))

#### Read the Delta Table as a Static or Streaming DataFrame
#### This dataframe will always be Up-To-Date

In [0]:
building_data = spark.read.format("delta").load("/delta/apartment/building_data").orderBy("Building_id")

In [0]:
display(building_data)

Building_id,Building_name,Landlord_id,Address_line_1,City,Post_code,Region,event_time,latest_event_time
1,Goodwin-Dickinson,1,8 American Ash Hill,Philadelphia,19151,Pennsylvania,2018-11-27T06:33:41.000+0000,2018-11-27T06:33:41.000+0000
2,Schultz-Schiller,2,37 Lotheville Hill,Dayton,45403,Ohio,2017-03-01T13:17:00.000+0000,2017-03-01T13:17:00.000+0000
3,Bashirian-Dibbert,3,45253 Gulseth Place,Denton,76210,Texas,2017-08-31T07:27:45.000+0000,2017-08-31T07:27:45.000+0000
4,Marquardt-Spencer,4,5590 Carpenter Junction,Muncie,47306,Indiana,2017-10-15T03:20:58.000+0000,2017-10-15T03:20:58.000+0000
5,"Ebert, Haley and Steuber",5,9354 Butternut Road,Orange,92668,California,2018-05-03T19:14:42.000+0000,2018-05-03T19:14:42.000+0000
6,"Hilll, Runte and Trantow",6,4 Crownhardt Road,Colorado Springs,80935,Colorado,2017-06-13T07:15:04.000+0000,2017-06-13T07:15:04.000+0000
7,Gottlieb LLC,7,993 International Way,Bradenton,34282,Florida,2017-11-07T22:51:57.000+0000,2017-11-07T22:51:57.000+0000
8,Bergstrom-Nitzsche,8,3 Sutteridge Place,Lafayette,47905,Indiana,2018-07-21T09:53:54.000+0000,2018-07-21T09:53:54.000+0000
9,"Jenkins, McCullough and Schimmel",9,2526 Cardinal Road,Jacksonville,32209,Florida,2018-02-02T22:50:38.000+0000,2018-02-02T22:50:38.000+0000
10,"Kutch, Mueller and Erdman",10,82416 Lindbergh Point,Indianapolis,46239,Indiana,2018-03-08T14:30:55.000+0000,2018-03-08T14:30:55.000+0000


### Do Some Live Streaming Graphs

In [0]:
building_data_stream = spark.readStream.format("delta").load("/delta/apartment/building_data")

In [0]:
display(building_data_stream.groupBy("Region").count())

Region,count
Utah,1
Minnesota,3
Ohio,3
Arkansas,1
Texas,9
Pennsylvania,3
Washington,2
Illinois,1
District of Columbia,1
Missouri,2
